# Stream Processing with Kafka

This notebook demonstrates real-time stream processing patterns using Kafka.

## Topics Covered:
- Real-time data aggregation
- Windowing operations
- Stateful processing
- Stream transformations
- Pattern detection

## 1. Setup and Configuration

In [ ]:
import os
import json
import time
from datetime import datetime, timedelta
from collections import defaultdict, deque
from kafka import KafkaProducer, KafkaConsumer
import threading

# Kafka cluster connection
KAFKA_SERVERS = os.getenv('KAFKA_BOOTSTRAP_SERVERS', 'kafka1:29092,kafka2:29093,kafka3:29094')
print(f"Connecting to Kafka at: {KAFKA_SERVERS}")

## 2. Setup Topics for Streaming

In [ ]:
from kafka.admin import KafkaAdminClient, NewTopic
from kafka.errors import TopicAlreadyExistsError

admin_client = KafkaAdminClient(bootstrap_servers=KAFKA_SERVERS.split(','))

topics = [
    NewTopic(name='sensor-data', num_partitions=3, replication_factor=2),
    NewTopic(name='alerts', num_partitions=2, replication_factor=2)
]

try:
    admin_client.create_topics(new_topics=topics, validate_only=False)
    print("✓ Topics created: sensor-data, alerts")
except TopicAlreadyExistsError:
    print("✓ Topics already exist")

admin_client.close()

## 3. Generate Simulated Sensor Data

In [ ]:
import random

def generate_sensor_data(producer, duration_seconds=30):
    """Generate simulated sensor readings"""
    sensor_ids = ['sensor-1', 'sensor-2', 'sensor-3', 'sensor-4']
    start_time = time.time()
    count = 0
    
    while time.time() - start_time < duration_seconds:
        sensor_id = random.choice(sensor_ids)
        
        # Generate data with occasional anomalies
        temperature = random.uniform(20, 30)
        if random.random() < 0.1:  # 10% chance of anomaly
            temperature = random.uniform(50, 70)
        
        data = {
            'sensor_id': sensor_id,
            'timestamp': datetime.now().isoformat(),
            'temperature': round(temperature, 2),
            'humidity': round(random.uniform(40, 80), 2),
            'pressure': round(random.uniform(1010, 1020), 2)
        }
        
        producer.send('sensor-data', value=data, key=sensor_id)
        count += 1
        time.sleep(0.5)  # Send every 500ms
    
    producer.flush()
    print(f"\n✓ Generated {count} sensor readings")

# Create producer
producer = KafkaProducer(
    bootstrap_servers=KAFKA_SERVERS.split(','),
    value_serializer=lambda v: json.dumps(v).encode('utf-8'),
    key_serializer=lambda k: k.encode('utf-8')
)

print("Starting data generation for 30 seconds...")
print("(This will run in background while we process)\n")

## 4. Real-Time Aggregation (Windowed Average)

In [ ]:
class WindowedAggregator:
    """Calculate moving average over time window"""
    
    def __init__(self, window_seconds=10):
        self.window_seconds = window_seconds
        self.data = defaultdict(deque)
    
    def add(self, sensor_id, value, timestamp):
        self.data[sensor_id].append((timestamp, value))
        self._cleanup(sensor_id, timestamp)
    
    def _cleanup(self, sensor_id, current_time):
        cutoff = current_time - timedelta(seconds=self.window_seconds)
        while self.data[sensor_id] and self.data[sensor_id][0][0] < cutoff:
            self.data[sensor_id].popleft()
    
    def get_average(self, sensor_id):
        if not self.data[sensor_id]:
            return None
        values = [v for _, v in self.data[sensor_id]]
        return sum(values) / len(values)
    
    def get_count(self, sensor_id):
        return len(self.data[sensor_id])

# Start data generation in background
generator_thread = threading.Thread(target=generate_sensor_data, args=(producer, 30))
generator_thread.daemon = True
generator_thread.start()

# Create consumer for stream processing
consumer = KafkaConsumer(
    'sensor-data',
    bootstrap_servers=KAFKA_SERVERS.split(','),
    value_deserializer=lambda m: json.loads(m.decode('utf-8')),
    auto_offset_reset='latest',
    group_id='stream-processor'
)

# Process stream with windowed aggregation
aggregator = WindowedAggregator(window_seconds=10)
start_time = time.time()
process_duration = 25

print("Processing sensor data stream...\n")

for message in consumer:
    data = message.value
    sensor_id = data['sensor_id']
    temperature = data['temperature']
    timestamp = datetime.fromisoformat(data['timestamp'])
    
    # Add to window
    aggregator.add(sensor_id, temperature, timestamp)
    
    # Calculate moving average
    avg_temp = aggregator.get_average(sensor_id)
    count = aggregator.get_count(sensor_id)
    
    print(f"{sensor_id}: Current={temperature:.1f}°C, 10s Avg={avg_temp:.1f}°C (n={count})")
    
    # Stop after duration
    if time.time() - start_time > process_duration:
        break

consumer.close()
print("\n✓ Stream processing completed")

## 5. Anomaly Detection (Pattern Recognition)

In [ ]:
class AnomalyDetector:
    """Detect temperature anomalies"""
    
    def __init__(self, threshold=40.0):
        self.threshold = threshold
        self.alert_producer = KafkaProducer(
            bootstrap_servers=KAFKA_SERVERS.split(','),
            value_serializer=lambda v: json.dumps(v).encode('utf-8')
        )
    
    def check(self, data):
        if data['temperature'] > self.threshold:
            alert = {
                'alert_type': 'HIGH_TEMPERATURE',
                'sensor_id': data['sensor_id'],
                'temperature': data['temperature'],
                'threshold': self.threshold,
                'timestamp': datetime.now().isoformat(),
                'severity': 'WARNING' if data['temperature'] < 60 else 'CRITICAL'
            }
            self.alert_producer.send('alerts', value=alert)
            return alert
        return None
    
    def close(self):
        self.alert_producer.close()

# Start fresh data generation
generator_thread2 = threading.Thread(target=generate_sensor_data, args=(producer, 20))
generator_thread2.daemon = True
generator_thread2.start()

time.sleep(1)  # Wait for generator to start

# Create consumer and detector
consumer2 = KafkaConsumer(
    'sensor-data',
    bootstrap_servers=KAFKA_SERVERS.split(','),
    value_deserializer=lambda m: json.loads(m.decode('utf-8')),
    auto_offset_reset='latest',
    group_id='anomaly-detector'
)

detector = AnomalyDetector(threshold=40.0)

print("Detecting anomalies (temperature > 40°C)...\n")

start_time = time.time()
alert_count = 0

for message in consumer2:
    data = message.value
    alert = detector.check(data)
    
    if alert:
        alert_count += 1
        print(f"🚨 ALERT: {alert['sensor_id']} - {alert['temperature']}°C ({alert['severity']})")
    else:
        print(f"✓ {data['sensor_id']}: {data['temperature']}°C - Normal")
    
    if time.time() - start_time > 15:
        break

detector.close()
consumer2.close()

print(f"\n✓ Detected {alert_count} anomalies")

## 6. Stream Enrichment (Join Pattern)

In [ ]:
# Sensor metadata (simulated lookup table)
sensor_metadata = {
    'sensor-1': {'location': 'Building A', 'floor': 1, 'room': 'Server Room'},
    'sensor-2': {'location': 'Building A', 'floor': 2, 'room': 'Office 201'},
    'sensor-3': {'location': 'Building B', 'floor': 1, 'room': 'Lab'},
    'sensor-4': {'location': 'Building B', 'floor': 3, 'room': 'Storage'}
}

def enrich_sensor_data(data):
    """Enrich sensor data with metadata"""
    sensor_id = data['sensor_id']
    metadata = sensor_metadata.get(sensor_id, {})
    return {**data, **metadata}

# Consume and enrich
consumer3 = KafkaConsumer(
    'sensor-data',
    bootstrap_servers=KAFKA_SERVERS.split(','),
    value_deserializer=lambda m: json.loads(m.decode('utf-8')),
    auto_offset_reset='earliest',
    group_id='enrichment-processor'
)

print("Enriching sensor data with metadata...\n")

count = 0
for message in consumer3:
    enriched = enrich_sensor_data(message.value)
    print(f"📍 {enriched['sensor_id']} @ {enriched.get('location', 'Unknown')} ")
    print(f"   {enriched.get('room', 'N/A')} - Temp: {enriched['temperature']}°C")
    print()
    
    count += 1
    if count >= 5:
        break

consumer3.close()
print("✓ Enrichment completed")

## 7. Stateful Processing (Session Tracking)

In [ ]:
class SessionTracker:
    """Track sensor sessions and calculate statistics"""
    
    def __init__(self):
        self.sessions = defaultdict(lambda: {
            'count': 0,
            'total_temp': 0,
            'min_temp': float('inf'),
            'max_temp': float('-inf'),
            'first_seen': None,
            'last_seen': None
        })
    
    def update(self, sensor_id, data):
        session = self.sessions[sensor_id]
        temp = data['temperature']
        timestamp = data['timestamp']
        
        session['count'] += 1
        session['total_temp'] += temp
        session['min_temp'] = min(session['min_temp'], temp)
        session['max_temp'] = max(session['max_temp'], temp)
        session['last_seen'] = timestamp
        
        if session['first_seen'] is None:
            session['first_seen'] = timestamp
    
    def get_stats(self, sensor_id):
        session = self.sessions[sensor_id]
        if session['count'] == 0:
            return None
        
        return {
            'sensor_id': sensor_id,
            'readings': session['count'],
            'avg_temp': session['total_temp'] / session['count'],
            'min_temp': session['min_temp'],
            'max_temp': session['max_temp'],
            'duration': session['last_seen']
        }

tracker = SessionTracker()

consumer4 = KafkaConsumer(
    'sensor-data',
    bootstrap_servers=KAFKA_SERVERS.split(','),
    value_deserializer=lambda m: json.loads(m.decode('utf-8')),
    auto_offset_reset='earliest',
    group_id='session-tracker'
)

print("Tracking sensor sessions...\n")

count = 0
for message in consumer4:
    data = message.value
    tracker.update(data['sensor_id'], data)
    count += 1
    
    if count >= 20:
        break

consumer4.close()

# Print session statistics
print("\nSession Statistics:\n")
for sensor_id in sorted(tracker.sessions.keys()):
    stats = tracker.get_stats(sensor_id)
    if stats:
        print(f"{stats['sensor_id']}:")
        print(f"  Readings: {stats['readings']}")
        print(f"  Avg Temp: {stats['avg_temp']:.1f}°C")
        print(f"  Range: {stats['min_temp']:.1f}°C - {stats['max_temp']:.1f}°C")
        print()

## 8. Cleanup

In [ ]:
# Close producer
producer.close()
print("✓ All resources cleaned up")

## Key Takeaways

1. **Windowing**: Use time-based or count-based windows for aggregations
2. **Stateful Processing**: Maintain state across messages for complex logic
3. **Stream Enrichment**: Join streaming data with reference data
4. **Pattern Detection**: Implement real-time anomaly detection
5. **Multiple Consumers**: Different consumers can process same stream differently

## Stream Processing Patterns

- **Filter**: Select specific messages based on criteria
- **Map**: Transform each message
- **Aggregate**: Combine multiple messages (windowed)
- **Join**: Combine data from multiple streams
- **Branch**: Route messages to different processors

## Production Considerations

- Use appropriate window sizes for your use case
- Implement checkpointing for fault tolerance
- Consider using Kafka Streams or Flink for complex processing
- Monitor consumer lag and processing latency